In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stopwords = set(stopwords.words('english'))

In [2]:
vox = pd.read_json('./data/voxtest.jsonl', lines=True)

In [3]:
trans = str.maketrans('', '', string.punctuation + '0123456789')
stemmer = SnowballStemmer('english')


def filter_func(document):
    result = []
    wordlist = nltk.word_tokenize(document.lower().translate(trans))
    for word in wordlist:
        c1 = word not in stopwords 
        c2 = len(word) > 2
        c3 = not word.startswith('https')
        c4 = not re.match('document[a-z]+', word)
        if c1 and c2 and c3 and c4:
            result.append(stemmer.stem(word.encode('ascii', 'ignore').decode('UTF-8')))
    return result

vox['id'] = vox._id.apply(lambda x: x['$oid'])
vox.drop('_id', axis=1, inplace=True)
vox.text = vox.text.apply(lambda x: filter_func(x))

In [4]:
y = vox.date.apply(lambda x: x[:-1] + 'AM' if x.endswith('a') else x[:-1] + 'PM' if x.endswith('p') else x)
y = y.apply(lambda x: x[:12] if x.endswith('M') else x)

for row in y.iteritems():
    try:
        if row[1] == 'NULL':
            y[row[0]] = np.nan
        else:
            y[row[0]] = pd.to_datetime(row[1])
    except:
        pass

vox.date = y

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
  ('tf', CountVectorizer()),
  ('lda', LatentDirichletAllocation())
])

parameters = {
  'lda__n_topics': (5, 6)
}

gs = GridSearchCV(pipeline, parameters)
gs.fit(vox.text.apply(lambda x: ' '.join(x)))


/Users/warren/Desktop/programming_projects/news_analysis/news_packages/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


/Users/warren/Desktop/programming_projects/news_analysis/news_packages/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


/Users/warren/Desktop/programming_projects/news_analysis/news_packages/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


/Users/warren/Desktop/programming_projects/news_analysis/news_packages/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


/Users/warren/Desktop/programming_projects/news_analysis/news_packages/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


/Users/warren/Desktop/programming_projects/news_analysis/news_packages/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


/Users/warren/Desktop/programming_projects/news_analysis/news_packages/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('tf', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_a...        random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'lda__n_topics': (5, 6)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring=None, verbose=0)

In [33]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-n_top_words - 1:-1]]))
  print()

feature_names = gs.best_estimator_.steps[0][1].get_feature_names()
lda = gs.best_estimator_.steps[1][1]
print_top_words(lda, feature_names, 4)

Topic #0:
trump like peopl one
Topic #1:
coal car oil california
Topic #2:
drug opioid heroin britney
Topic #3:
health abort insur contracept
Topic #4:
mar weiner earth human



In [ ]:
cnt_vct = CountVectorizer()
tf = cnt_vct.fit_transform(vox.text.apply(lambda x: ' '.join(x)))



def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-n_top_words - 1:-1]]))
  print()

lda = LatentDirichletAllocation(n_topics=12, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)
tf_feature_names = cnt_vct.get_feature_names()
print_top_words(lda, tf_feature_names, 8)